In [14]:
import sklearn
from sklearn import datasets
import pandas as pd

In [62]:
from sklearn.model_selection import cross_val_score
def load_data(data, input_features, target_feature, cat_features=None, num_features=None, txt_features=None):
    all_features = input_features + [target_feature]
    print(data.values.shape)
    data = data[all_features]
    data.dropna(subset=[target_feature], inplace=True)

    # change categorical features to numeric code
    if(cat_features!=None):
        data[cat_features] = data[cat_features].astype('category')
        data[cat_features] = data[cat_features].apply(lambda x: x.cat.codes)
    # replace nan with 0 in numerical features
    if(num_features!=None):
        data[num_features] = data[num_features].fillna(0.)
        for feature in num_features:
            data[feature] = data[feature].apply(lambda x: replace_string(x))
    if(txt_features!=None):
        if txt_features:
            data[txt_features] = data[txt_features].fillna('')

    return data


In [46]:
testdata=sklearn.datasets.fetch_20newsgroups(data_home=None, subset='test', categories=None, shuffle=True, random_state=42, remove=(['headers', 'footers', 'quotes']), download_if_missing=True)
data=sklearn.datasets.fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(['headers', 'footers', 'quotes']), download_if_missing=True)

In [27]:
data = pd.DataFrame(list(zip(data['data'],data['target'])))

In [29]:
data.columns = ['text','category']

In [ ]:
#lowercase
data['text'] = [data.lower() for data in data['text']]

In [ ]:
#stopwords
stop_words = set(stopwords.words(‘english’))
from nltk.tokenize import word_tokenize
data['text'] = [word_tokenize(data) for data in data['text'] word_tokenize(input_str)]
result = [i for i in tokens if not i in stop_words]

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
#Count vectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data.data)
print(X_train_counts.shape)
#counts
count_vect.vocabulary_

(11314, 130107)

In [57]:
#frequencies
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(11314, 130107)

In [64]:
model.get_depth()

197

In [93]:
def loguniform(low=0, high=1, size=None):
    return stats.reciprocal(np.exp(low), np.exp(high))

In [182]:
def randomizedsearch(X,y,model=DecisionTreeClassifier(random_state=0,criterion ='gini'),
                     n_iter_search=20,
                      param_dist = {'splitter':['best','random'],
                                    'max_features':["auto","sqrt","log2",None],
                                    'class_weight': ['balanced',None],
                                    'min_samples_leaf':[1,5,10,25,50,100],
                                    'min_samples_split':[2,5,10,25,50,100],
                                    'max_depth':[10,25,50,100,200]}):
    random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search)
    start = time()
    random_search.fit(X_train_tf, data.target)
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
    report(random_search.cv_results_)
    return(random_search)
    
def gridsearch(X,y,model=DecisionTreeClassifier(random_state=0,criterion ='gini'),
               param_grid = {'splitter':['best','random'],
                                    'max_features':["auto","sqrt","log2",None],
                                    'class_weight': ['balanced',None],
                                    'min_samples_leaf':[1,5,10,25,50,100],
                                    'min_samples_split':[2,5,10,25,50,100],
                                    'max_depth':[10,25,50,100,200]}):
    grid_search = GridSearchCV(model, param_grid=param_grid)
    start = time()
    grid_search.fit(X, y)
    print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
    report(grid_search.cv_results_)
    return(grid_search)

In [183]:
rs=randomizedsearch(X_train_tf,data.target)

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV took 125.88 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.613 (std: 0.006)
Parameters: {'splitter': 'best', 'min_samples_split': 25, 'min_samples_leaf': 5, 'max_features': None, 'max_depth': 100, 'class_weight': None}

Model with rank: 2
Mean validation score: 0.595 (std: 0.014)
Parameters: {'splitter': 'random', 'min_samples_split': 100, 'min_samples_leaf': 10, 'max_features': None, 'max_depth': 100, 'class_weight': 'balanced'}

Model with rank: 3
Mean validation score: 0.519 (std: 0.007)
Parameters: {'splitter': 'best', 'min_samples_split': 100, 'min_samples_leaf': 50, 'max_features': None, 'max_depth': 50, 'class_weight': 'balanced'}



In [172]:
gridsearch(X_train_tf,data.target)

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV took 10770.57 seconds for 2880 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.635 (std: 0.017)
Parameters: {'class_weight': 'balanced', 'max_depth': 200, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 50, 'splitter': 'random'}

Model with rank: 2
Mean validation score: 0.632 (std: 0.007)
Parameters: {'class_weight': None, 'max_depth': 200, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 50, 'splitter': 'random'}

Model with rank: 3
Mean validation score: 0.632 (std: 0.007)
Parameters: {'class_weight': None, 'max_depth': 200, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 100, 'splitter': 'random'}



In [103]:
model.get_params().keys()

dict_keys(['class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])